# 프로젝트: 멋진 작사가 만들기

## Step 1. 데이터 다운로드
<hr>
이미 실습(1) 데이터 다듬기에서 Cloud shell에 심볼릭 링크로 ~/aiffel/lyricist/data를 생성하셨다면, ~/aiffel/lyricist/data/lyrics에 데이터가 있습니다.

## Step 2. 데이터 읽어오기
<hr>
glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [6]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I hear you callin\', "Here I come baby"', 'To save you, oh oh', "Baby no more stallin'"]


## Step 3. 데이터 정제
<hr>
앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 
너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

### 1) 데이터 정제용 함수

In [82]:
import os, re 
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


### 2) 공백 및 불필요한 문장 제거 

In [83]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:

    # 공백을 제거한 데이터가 없으면 건너뜀
    if len(sentence.strip())==0 : continue
            
    # 키워드 정제
    preprocessed_sentence = preprocess_sentence(sentence)

    #  정제 후 토큰화한 개수가 15개 이상은 건너뜀
    if len(preprocessed_sentence.split()) > 15: continue

    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> i hear you callin , here i come baby <end>',
 '<start> to save you , oh oh <end>',
 '<start> baby no more stallin <end>',
 '<start> these hands have been longing to touch you baby <end>',
 '<start> and now that you ve come around , to seein it my way <end>',
 '<start> it s unbelieveable how your body s calling for me <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you tell me , what s your desire <end>',
 '<start> baby your wish is my deal oh yes it is baby <end>']

## Step 4. 평가 데이터셋 분리
<hr>
훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

### 1)  tokenize로 텐서 변환

In [84]:
import numpy as np
import tensorflow as tf

# 12000장의 데이터로 설정
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
tensor.shape

[[  2   4 186 ...   0   0   0]
 [  2  10 588 ...   0   0   0]
 [  2  52  41 ...   0   0   0]
 ...
 [  2   4  92 ...   0   0   0]
 [  2   9 156 ...   0   0   0]
 [  2 178  16 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fb41257f090>


(156201, 15)

### 2) train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리

In [85]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    
print(src_input[0])
print(tgt_input[0])
print(src_input.shape)
print(tgt_input.shape)

[  2   4 186   7 824   5  90   4  68  52   3   0   0   0]
[  4 186   7 824   5  90   4  68  52   3   0   0   0   0]
(156201, 14)
(156201, 14)


In [86]:
from sklearn.model_selection import train_test_split # train_test_split 모듈 import
enc_train, enc_val, dec_train, dec_val = train_test_split( src_input,
                                                           tgt_input,
                                                           test_size=0.2, 
                                                           random_state=21)

In [87]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124960, 14)
Target Train: (124960, 14)


**아래 값과 같은지 확인**
```
out:

Source Train: (124960, 14)
Target Train: (124960, 14)
```

### 3) 데이터 셋 만들기

In [72]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기
<hr>
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! 
(Loss는 아래 제시된 Loss 함수를 그대로 사용!)

그리고 멋진 모델이 생성한 가사 한 줄을 제출하시길 바랍니다!

### 1) 모델 생성 및 훈련

In [73]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
609/609 [==============================] - 214s 343ms/step - loss: 3.9147
Epoch 2/10
609/609 [==============================] - 212s 348ms/step - loss: 3.0057
Epoch 3/10
609/609 [==============================] - 212s 348ms/step - loss: 2.8035
Epoch 4/10
609/609 [==============================] - 212s 348ms/step - loss: 2.6577
Epoch 5/10
609/609 [==============================] - 211s 347ms/step - loss: 2.5385
Epoch 6/10
609/609 [==============================] - 211s 346ms/step - loss: 2.4326
Epoch 7/10
609/609 [==============================] - 211s 346ms/step - loss: 2.3375
Epoch 8/10
609/609 [==============================] - 210s 345ms/step - loss: 2.2467
Epoch 9/10
609/609 [==============================] - 209s 342ms/step - loss: 2.1593
Epoch 10/10
609/609 [==============================] - 208s 341ms/step - loss: 2.0762


### 2) 테스트 수행

In [76]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

데이터가 커서 훈련하는 데 시간이 제법 걸릴 겁니다. 여유를 가지고 작업하시면 좋아요 :)

In [91]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna be <end> '

Q4. 모델이 생성한 가사 한 줄을 제출하세요.

'<start> i love you , i m not gonna be <end> '

## 회고

### 1. 이번 프로젝트에서 어려웠던 점,

* tokenize로 텐서 변환 후, 데이터 건 수를 124,960 건 수를 맞추기 위해서 힘들었음
딱 특정 케이스에만 해당 건수가 나와서 은근히 시간이 걸렸음

### 2. 프로젝트를 진행하면서 알아낸 점 혹은 아직 모호한 점.
알아낸점
* 텍스트에 대한 글짓기를 어떤 방식으로 하는지 알게 되었음

모호한점
* 대략적인 함수들이 어떤일은 하는지는 알것 같으나, 각각이 세부적으로 어떤일을 하는지는 아직도 잘 이해가 안 감

### 3. 루브릭 평가 지표를 맞추기 위해 시도한 것들.
* 데이터 전처리부분에서 건수를 동일하게 맞추려고 다양한 방식으로 시도했음
 ( trim을 하고 길이를 재거나.. null 체크를 하는 위치 방식 등등.. )


### 4. 만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정.
* 건수도 달성하고, 모델이 생성한 가사 한 줄도 제출하긴 했는데..
마지막 gonna be.. 뒷 부분이 안 나온게 좀 아쉬웠음
뭔가 더 조정했으면 뒤에 마지막 가사까지 만들었으려나.. 

전체적으로 이미지 다루는 것보다는 재미있었음 +_+